In [13]:
import pandas as pd
import os
import fnmatch
import numpy as np
from library import start
from library import clean_tea

In [14]:
start.data_path

'/Users/kylieleblancKylie/domino/dofis/data/'

In [62]:
year = 'yr1213'

# Certifications

In [16]:
folder = 'certification_' + year + '/'
teacher_datapath = os.path.join(start.data_path, 'tea', 'teachers', folder)

pattern = "CERTIFICATION*.csv"
if year == 'yr1213' or year == 'yr1314':
    pattern = "CERTIFICATION*.TXT"
cert_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        cert_files.append(entry)
cert_files.sort()
dirs_cert = [teacher_datapath + file for file in cert_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_cert]
certification = pd.concat(df_list)
certification.head()

,PERSONID_SCRAM,DISTRICT,REGION,CERTIFICATE_ID_NUMBER,CERTIFICATE_LIFE,CREDENTIAL_TYPE,CERTIFICATE_PREPARATION_ROUTE,CERTIFICATE_EFFECTIVE_DATE,CERTIFICATE_EXPIRATION_DATE,SUBJECT_AREA,...,CERTIFICATION_LEVEL,ROLE_CREDENTIALED_FOR,POPULATION_CREDENTIALED_FOR,PREPARATION_PROGRAM,EMAIL_ADDRESS,MAILING_ADDRESS,CITY,ST,ZIP_CD,PHONE
0,*30*5P341,031901,01,2021478,LIFE,Paraprofessional,Paraprofessional Program,06OCT1994:00:00:00,NaN,Other,...,Not Applicable,Educational Aide,Not Applicable,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
1,*30*5P341,031901,01,2502767,SYR,Emergency Non-Certified,Permit Program,09JAN2002:00:00:00,31AUG2002:00:00:00,Special Education,...,Special Education,Teacher,Special Education,BROWNSVILLE ISD,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
2,*30*5P341,031901,01,2595245,1YR,Probationary,Alternative Program,21AUG2002:00:00:00,21AUG2003:00:00:00,Special Education,...,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
3,*30*5P341,031901,01,2744684,1YR,Probationary Extension,Alternative Program,31AUG2003:00:00:00,31AUG2004:00:00:00,Special Education,...,Special Education,Teacher,Special Education,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966
4,*30*5P341,031901,01,2922393,5YR,Standard,Alternative Program,02APR2005:00:00:00,31OCT2010:00:00:00,Special Education,...,Special Education,Teacher,Regular Students,REG I EDUCATION SERVICE CENTER,nagandara@bisd.us,1413 MISSION BEND,Brownsville,TX,78526,9563723966


In [17]:
vars_to_keep = {'PERSONID_SCRAM': 'teacher_id', 'DISTRICT': 'district',
                'CREDENTIAL_TYPE': 'cert_type', 'CERTIFICATE_PREPARATION_ROUTE': 'cert_route',
                'CERTIFICATION_LEVEL': 'cert_level', 'CREDENTIALED_GRADES': 'cert_grades',
                'SUBJECT_AREA': 'cert_area', 'SUBJECT': 'cert_subject'}
certification = clean_tea.filter_and_rename_cols(certification, vars_to_keep)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education


In [18]:
cert_types = {'Emergency Non-Certified': False, 'Emergency Certified': False, 
              'Emergency': False, 'Emergency Teaching': False,
              'Temporary Exemption': False, 'Temporary Teaching Certificate': False,
              'Unknown Permit': False, 'Unknown': False,
              'Special Assignment': False,
              'Paraprofessional': False, 'Standard Paraprofessional': False,
              'Standard': True, 'Provisional': True,
              'Probationary': True, 'Probationary Extension': True, 'Probationary Second Extension': True,
              'Non-renewable': True, 'One Year': True,
              'Visiting International Teacher': True,
              'Professional': True, 'Standard Professional':True}
certification['certification'] = certification['cert_type'].map(cert_types)
certification['vocational'] = np.where((certification['cert_type'] == "Vocational"), True, False)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False


In [19]:
grades_elem = {'Grades EC-4': True, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': True, 'Grades PK-3': True, 'Grades PK-5': True,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': False,'Grades 6-10': False, 'Grades 6-12': False,
               'Grades 7-12': False,
               'Grades 8-12': False}
grades_middle = {'Grades EC-4': False, 'Grades EC-6': True,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': True,'Grades 1-6': True,
               'Grades 4-8': True,
               'Grades 6-8': True,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
grades_high = {'Grades EC-4': False, 'Grades EC-6': False,'Grades EC-12': True,
               'Grades PK-KG': False, 'Grades PK-3': False, 'Grades PK-5': False,'Grades PK-12': True,
               'Grades 1-8': False,'Grades 1-6': False,
               'Grades 4-8': False,
               'Grades 6-8': False,'Grades 6-10': True, 'Grades 6-12': True,
               'Grades 7-12': True,
               'Grades 8-12': True}
certification['cert_elem'] = certification['cert_grades'].map(grades_elem)
certification['cert_elem'] = np.where((certification.certification == False), False, certification.cert_elem) #TODO code review

certification['cert_middle'] = certification['cert_grades'].map(grades_middle)
certification['cert_middle'] = np.where((certification.certification == False), False, certification.cert_middle)

certification['cert_high'] = certification['cert_grades'].map(grades_high)
certification['cert_high'] = np.where((certification.certification == False), False, certification.cert_high)

certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational,cert_elem,cert_middle,cert_high
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False,False,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False,False,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False,True,True,True


In [20]:
certification['cert_area_elem'] = np.where(certification['cert_area'] == "General Elementary (Self-Contained)", True, False)
certification['cert_area_ela'] = np.where(certification['cert_area'] == "English Language Arts", True, False)
certification['cert_area_math'] = np.where(certification['cert_area'] == "Mathematics", True, False)
certification['cert_area_sci'] = np.where(certification['cert_area'] == "Science", True, False)
certification.head()

,teacher_id,district,cert_type,cert_route,cert_level,cert_grades,cert_area,cert_subject,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
0,*30*5P341,031901,Paraprofessional,Paraprofessional Program,Not Applicable,NaN,Other,Not Applicable,False,False,False,False,False,False,False,False,False
1,*30*5P341,031901,Emergency Non-Certified,Permit Program,Special Education,Grades PK-12,Special Education,Generic Special Education,False,False,False,False,False,False,False,False,False
2,*30*5P341,031901,Probationary,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False
3,*30*5P341,031901,Probationary Extension,Alternative Program,Special Education,Grades PK-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False
4,*30*5P341,031901,Standard,Alternative Program,Special Education,Grades EC-12,Special Education,Generic Special Education,True,False,True,True,True,False,False,False,False


In [21]:
certification = certification[['teacher_id', 'district', 'cert_level', 'cert_area', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]
certification.sample(n=10)

,teacher_id,district,cert_level,cert_area,certification,vocational,cert_elem,cert_middle,cert_high,cert_area_elem,cert_area_ela,cert_area_math,cert_area_sci
70199,F3Q450341,079907,Elementary,General Elementary (Self-Contained),True,False,True,False,False,True,False,False,False
11335,Q34Q3P043,043902,All Level,Health and Physical Education,True,False,True,True,True,False,False,False,False
355087,P323V*546,101913,Elementary,General Elementary (Self-Contained),True,False,True,True,False,True,False,False,False
71695,D3743*349,227901,Secondary,Mathematics,True,False,False,True,True,False,False,True,False
153194,Y33270346,220902,Supplemental,Other,True,False,NaN,NaN,NaN,False,False,False,False
190442,V35Q2QV44,220905,Elementary,Bilingual Education,False,False,False,False,False,False,False,False,False
419360,V3331V*46,101915,Endorsement,Special Education,True,False,True,True,True,False,False,False,False
378897,D3525*248,234907,Secondary,Fine Arts,True,False,False,True,True,False,False,False,False
40241,D34Q4*Q45,251901,Elementary,General Elementary (Self-Contained),True,False,True,True,False,True,False,False,False
89531,D3305Q448,015912,Not Applicable,Other,False,False,False,False,False,False,False,False,False


## Collapse to teacher level

In [22]:
teacher_cert = certification[['teacher_id', 'district', 'certification', 'vocational', 
                                 'cert_elem', 'cert_middle', 'cert_high',
                                 'cert_area_elem', 'cert_area_ela', 'cert_area_math', 'cert_area_sci']]
teacher_cert.groupby(['teacher_id', 'district']).max()
teacher_cert.sample(n = 15)

KeyboardInterrupt: 

## Sort and to csv

In [11]:
teacher_cert.sort_values(by = ['teacher_id'], axis = 0)
filename = 'teacher_cert' + year + '.csv'
certification.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))

# Teacher Assignments

In [63]:
pattern = "TEACHER_MASTER*.TXT"
teacher_files = []
for entry in os.listdir(teacher_datapath):
    if fnmatch.fnmatch(entry, pattern):
        teacher_files.append(entry)
teacher_files.sort()
dirs_teachers = [teacher_datapath + file for file in teacher_files]
df_list = [pd.read_csv(file, sep=",", encoding="ISO-8859-1", dtype=object) for file in dirs_teachers]
teachers = pd.concat(df_list)
teachers = teachers[teachers['ROLE NAME'] == 'TEACHER']

In [64]:
pd.set_option('display.max_columns', 100)
teachers

,REGION NUMBER,COUNTY NUMBER,COUNTY NAME,DISTRICT NUMBER,DISTRICT NAME,DISTRICT TYPE CODE,DISTRICT TYPE NAME,DISTRICT CHARTER TYPE CODE,DISTRICT CHARTER TYPE NAME,DISTRICT CATEGORY CODE,DISTRICT CATEGORY NAME,DISTRICT PHONE,DISTRICT FAX,DISTRICT ADDRESS,DISTRICT CITY,DISTRICT STATE,DISTRICT ZIP,CAMPUS NAME,CAMPUS LOW GRADE,CAMPUS HIGH GRADE,CAMPUS TYPE CODE,CAMPUS TYPE NAME,CAMPUS CHARTER TYPE CODE,CAMPUS CHARTER TYPE NAME,CAMPUS GRADE GROUP CODE,CAMPUS GRADE GROUP NAME,CAMPUS ADDRESS,CAMPUS CITY,CAMPUS STATE,CAMPUS ZIP,CAMPUS PHONE,SCRAMBLED UNIQUE ID,FIRST NAME,MIDDLE NAME,LAST NAME,SEX CODE,SEX NAME,ETHNICITY CODE,ETHNICITY NAME,EXPERIENCE,DEGREE CODE,DEGREE NAME,FTE,BASE PAY,OTHER PAY,TOTAL PAY,ROLE CODE,ROLE NAME,ROLE FULL TIME EQUIVALENT,ROLE BASE PAY,SUBJECT AREA CODE 1,SUBJECT AREA NAME 1,SUBJECT AREA CODE 2,SUBJECT AREA NAME 2,SUBJECT AREA CODE 3,SUBJECT AREA NAME 3,SUBJECT AREA CODE 4,SUBJECT AREA NAME 4,SUBJECT AREA CODE 5,SUBJECT AREA NAME 5,SUBJECT AREA CODE 6,SUBJECT AREA NAME 6,SUBJECT AREA CODE 7,SUBJECT AREA NAME 7,SUBJECT AREA CODE 8,SUBJECT AREA NAME 8,SUBJECT AREA CODE 9,SUBJECT AREA NAME 9,SUBJECT AREA CODE 10,SUBJECT AREA NAME 10,PAY TYPE CODE 1,PAY TYPE NAME 1,PAY TYPE CODE 2,PAY TYPE NAME 2,PAY TYPE CODE 3,PAY TYPE NAME 3,PAY TYPE CODE 4,PAY TYPE NAME 4,PAY TYPE CODE 5,PAY TYPE NAME 5,CAMPUS NUMBER
0,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,3,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,SHARP EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,1439 PALM BLVD,BROWNSVILLE,TX,78520-7292,(956)982-2930,*30*5P341,NORMA,ALICIA,GANDARA,F,FEMALE,H,Hispanic/Latino,14,1,BACHELOR'S,1.0000,48900,3500,52400,087,TEACHER,1.0000,48900,99,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79,OTHER SUPPLEMENTAL DUTIES,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,031901111
1,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,3,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,OLIVEIRA MIDDLE,06,08,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,2,MIDDLE SCHOOL,444 LAND O' LAKES,BROWNSVILLE,TX,78521-4296,(956)548-8530,*3034L040,JOSE,NaN,REBOLLO,M,MALE,H,Hispanic/Latino,12,2,MASTER'S,1.0000,51194,0,51194,087,TEACHER,1.0000,51195,09,TECHNOLOGY APPLICATIONS,11,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901045
2,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,3,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,GARDEN PARK EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,855 MILITARY HWY,BROWNSVILLE,TX,78520-4661,(956)982-2630,*3040V545,ANNA,M,ORTEGA,F,FEMALE,H,Hispanic/Latino,30,2,MASTER'S,1.0000,63185,0,63185,087,TEACHER,1.0000,63185,02,ENGLISH LANGUAGE ARTS,04,SCIENCE,05,SOCIAL STUDIES,09,TECHNOLOGY APPLICATIONS,99,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901106
3,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,3,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-2417,DEL CASTILLO EL,EE,05,1,INSTRUCTIONAL CAMPUS,00,NOT A CHARTER SCHOOL,1,ELEMENTARY,105 MORNINGSIDE RD,BROWNSVILLE,TX,78521-4932,(956)982-2600,*3100Y546,GRACIELA,NaN,VASQUEZ,F,FEMALE,H,Hispanic/Latino,14,1,BACHELOR'S,1.0000,48900,0,48900,087,TEACHER,1.0000,48900,99,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,BASE SALARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,031901120
4,01,031,CAMERON COUNTY,031901,BROWNSVILLE ISD,L,INDEPENDENT SCHL DIST(901-939),00,NOT A CHARTER DISTRICT,3,OTHER CENTRAL CITY,(956)548-8000,(956)574-6497,1900 PRICE RD,BROWNSVILLE,TX,78521-241

In [25]:
vars_to_keep = {'SCRAMBLED UNIQUE ID': 'teacher_id', 'DISTRICT NUMBER': 'district', 'DISTRICT NAME': 'distname',
                        'CAMPUS NUMBER': 'campus', 'CAMPUS NAME': 'campname',
                        'CAMPUS GRADE GROUP NAME': 'camp_grade_group',
                        'FTE': 'fte', 'ROLE FULL TIME EQUIVALENT': 'fte_teacher',
                        'SUBJECT AREA NAME 1': 'sub_area1', 'SUBJECT AREA NAME 2': 'sub_area2',
                        'SUBJECT AREA NAME 3': 'sub_area3',
                        'SUBJECT AREA NAME 4': 'sub_area4', 'SUBJECT AREA NAME 5': 'sub_area5'}
teachers = clean_tea.filter_and_rename_cols(teachers, vars_to_keep)
teachers.head()

,teacher_id,district,distname,campus,campname,camp_grade_group,fte,fte_teacher,sub_area1,sub_area2,sub_area3,sub_area4,sub_area5
0,*30*5P341,031901,BROWNSVILLE ISD,031901111,SHARP EL,ELEMENTARY,1.0000,1.0002,SELF-CONTAINED,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES
1,*3034L040,031901,BROWNSVILLE ISD,031901045,OLIVEIRA MIDDLE,MIDDLE SCHOOL,1.0000,1.0000,CAREER & TECHNOLOGY EDUCATION,NaN,NaN,NaN,NaN
2,*3040V545,031901,BROWNSVILLE ISD,031901106,GARDEN PARK EL,ELEMENTARY,1.0000,0.9999,ENGLISH LANGUAGE ARTS,MATHEMATICS,SCIENCE,SOCIAL STUDIES,PHYSICAL ED. & HEALTH
3,*3106V240,031901,BROWNSVILLE ISD,031901129,MORNINGSIDE EL,ELEMENTARY,1.0000,0.0496,OTHER,NaN,NaN,NaN,NaN
4,*31271Q42,031901,BROWNSVILLE ISD,031901003,PACE EARLY COLLEGE H S,HIGH SCHOOL,1.0000,0.6426,MATHEMATICS,SOCIAL STUDIES,PHYSICAL ED. & HEALTH,FOREIGN LANGUAGE,NaN


In [37]:
teachers.sub_area1.value_counts()

ENGLISH LANGUAGE ARTS            111213
SELF-CONTAINED                    51742
MATHEMATICS                       42564
NOT APPLICABLE                    26686
SCIENCE                           24477
PHYSICAL ED. & HEALTH             22294
SOCIAL STUDIES                    21670
FINE ARTS                         19666
CAREER & TECHNOLOGY EDUCATION     14664
SPECIAL EDUCATION                 10626
OTHER                              7581
FOREIGN LANGUAGE                   6883
TECHNOLOGY APPLICATIONS            1947
Name: sub_area1, dtype: int64

In [28]:
teachers.camp_grade_group.value_counts()

ELEMENTARY                166416
HIGH SCHOOL               102949
MIDDLE SCHOOL              62879
ELEMENTARY / SECONDARY     16878
JUNIOR HIGH SCHOOL         11483
OTHER GRADE GROUP           1408
Name: camp_grade_group, dtype: int64

In [36]:
teachers['campus_elem'] = np.where((teachers.camp_grade_group == "ELEMENTARY") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)
teachers['campus_middle'] = np.where((teachers.camp_grade_group == "MIDDLE SCHOOL") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)
teachers['campus_high'] = np.where((teachers.camp_grade_group == "HIGH SCHOOL") | (teachers.camp_grade_group == "ELEMENTARY / SECONDARY"), True, False)

In [48]:
teachers['course_elem'] = np.where(((teachers.sub_area1 == "SELF-CONTAINED") |
                                     (teachers.sub_area2 == "SELF-CONTAINED") |
                                     (teachers.sub_area3 == "SELF-CONTAINED") |
                                     (teachers.sub_area4 == "SELF-CONTAINED") |
                                     (teachers.sub_area5 == "SELF-CONTAINED")), True, False)
teachers['course_math'] = np.where(((teachers.sub_area1 == "MATHEMATICS") |
                                     (teachers.sub_area2 == "MATHEMATICS") |
                                     (teachers.sub_area3 == "MATHEMATICS") |
                                     (teachers.sub_area4 == "MATHEMATICS") |
                                     (teachers.sub_area5 == "MATHEMATICS")), True, False)
teachers['course_science'] = np.where(((teachers.sub_area1 == "SCIENCE") |
                                     (teachers.sub_area2 == "SCIENCE") |
                                     (teachers.sub_area3 == "SCIENCE") |
                                     (teachers.sub_area4 == "SCIENCE") |
                                     (teachers.sub_area5 == "SCIENCE")), True, False)
teachers['course_cte'] = np.where(((teachers.sub_area1 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area2 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area3 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area4 == "CAREER & TECHNOLOGY EDUCATION") |
                                     (teachers.sub_area5 == "CAREER & TECHNOLOGY EDUCATION")), True, False)

In [50]:
teachers = teachers[['teacher_id', 'district', 'campus', 'campus_elem', 'campus_middle', 'campus_high', 'course_elem', 'course_math', 'course_science', 'course_cte']]
teachers.head()

,teacher_id,district,campus,campus_elem,campus_middle,campus_high,course_elem,course_math,course_science,course_cte
0,*30*5P341,031901,031901111,True,False,False,True,True,True,False
1,*3034L040,031901,031901045,False,True,False,False,False,False,True
2,*3040V545,031901,031901106,True,False,False,False,True,True,False
3,*3106V240,031901,031901129,True,False,False,False,False,False,False
4,*31271Q42,031901,031901003,False,False,True,False,True,False,False


In [51]:
teachers.sort_values(by = ['teacher_id'], axis = 0)
filename = 'teacher_course_' + year + '.csv'
teachers.to_csv(os.path.join(start.data_path, 'tea', 'teachers', filename))